In [4]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier

In [34]:
#with open("syms.txt","r") as f:
#  lines = f.readlines()
#  f.close()
#lst = []
#for line in lines:
#  line = line.strip()
#  lst.append(line)
#symbols = lst[:25]

screens = ['all_cryptocurrencies_au',
 'all_cryptocurrencies_ca',
 'all_cryptocurrencies_eu',
 'all_cryptocurrencies_gb',
 'all_cryptocurrencies_in',
 'all_cryptocurrencies_us']
s = Screener()
symbols = []
for i in screens:
   data = s.get_screeners(i, count=250)
   dicts = data[i]['quotes']
   syms = [d['symbol'] for d in dicts]
   for sym in syms:
     symbols.append(sym)
print(len(symbols))

1500


In [35]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","GGPI","AI","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]

dfs = []

for i in symbols:
   data = yf.download(i,period="Max",interval="1d")
   if data.empty :
     print("Passing...")
   else:
        dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
    if df.shape[0] > 21:
       dfs.append(df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [36]:
fixed_dfs = []
for df in dfs:
    fixed_df = []
    df = np.array(df)
    for i in range(20, df.shape[0]-1):
        twenty_days_ago = i-20
        ninteen_days_ago = i-19
        eighteen_days_ago = i-18
        seventeen_days_ago = i-17
        sixteen_days_ago = i-16
        fifteen_days_ago = i-15
        fourteen_days_ago = i-14
        thirteen_days_ago = i-13
        twelve_days_ago = i-12
        eleven_days_ago = i-11
        ten_days_ago = i-10
        nine_days_ago = i-9
        eight_days_ago = i-8
        seven_days_ago = i-7
        six_days_ago = i-6
        five_days_ago = i-5
        four_days_ago = i-4
        three_days_ago = i-3
        two_days_ago = i - 2
        yesterday = i - 1
        today = i
        tomorrow = i + 1
        if df[tomorrow][3] > df[today][3]:
            future = 1
        else:
            future = 0


        row = [df[twenty_days_ago][3],df[ninteen_days_ago][3],df[eighteen_days_ago][3],df[seventeen_days_ago][3],df[sixteen_days_ago][3],df[fifteen_days_ago][3],df[fourteen_days_ago][3],df[thirteen_days_ago][3],df[twelve_days_ago][3],df[eleven_days_ago][3],
                   df[ten_days_ago][3], df[nine_days_ago][3], df[eight_days_ago][3], df[seven_days_ago][3], df[six_days_ago][3],
                   df[five_days_ago][3], df[four_days_ago][3], df[three_days_ago][3], df[two_days_ago][3], df[yesterday][3],
                   df[today][3], future]

        fixed_df.append(row)


    arrayed_fixed_df = np.array([fixed_df])
    two_d_fixed_df = []
    for i in range(0, arrayed_fixed_df.shape[1]):
       two_d_fixed_df.append(arrayed_fixed_df[0][i])
    
    mm = np.array(two_d_fixed_df)


    column_names = ["twenty_days_ago","ninteen_days_ago","eighteen_days_ago","seventeen_days_ago","sixteen_days_ago","fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today", "result"]
    
    column_names_without_result = ["twenty_days_ago","ninteen_days_ago","eighteen_days_ago","seventeen_days_ago","sixteen_days_ago","fifteen_days_ago","fourteen_days_ago","thirteen_days_ago","twelve_days_ago","eleven_days_ago",
                      "ten_days_ago", "nine_days_ago", "eight_days_ago", "seven_days_ago", "six_days_ago",
                       "five_days_ago", "four_days_ago", "three_days_ago", "two_days_ago", "yesterday", 
                        "today"]
    dff = pd.DataFrame(mm, columns=column_names)
    scaler = MinMaxScaler()
    result = dff["result"]
    dff = dff.drop(["result"],axis=1)
    scaled = pd.DataFrame(scaler.fit_transform(dff.T).T, columns=column_names_without_result,dtype=object)

    scaled["result"] = result
       
    fixed_dfs.append(scaled)

data = fixed_dfs[0]
for i in range(1,len(dfs)):
  data = pd.concat([data,fixed_dfs[i]], ignore_index = True)

data = data.astype(float)
data = data.dropna()
X = data.drop(["result"],axis=1)
y = data["result"]
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=99)
print(xTrain.shape,end=" ")
print(yTrain.shape)
print(xTest.shape,end=" ")
print(yTest.shape)

(1567966, 21) (1567966,)
(174219, 21) (174219,)


In [37]:
data

,twenty_days_ago,ninteen_days_ago,eighteen_days_ago,seventeen_days_ago,sixteen_days_ago,fifteen_days_ago,fourteen_days_ago,thirteen_days_ago,twelve_days_ago,eleven_days_ago,...,eight_days_ago,seven_days_ago,six_days_ago,five_days_ago,four_days_ago,three_days_ago,two_days_ago,yesterday,today,result
0,1.000000,0.729734,0.513487,0.625902,0.542358,0.590513,0.871146,0.766122,0.693372,0.650522,...,0.433655,0.518331,0.489081,0.402213,0.316233,0.064890,0.000000,0.045812,0.082981,1.0
1,0.837672,0.589439,0.718481,0.622581,0.677858,1.000000,0.879442,0.795931,0.746743,0.701039,...,0.595000,0.561423,0.461706,0.363009,0.074488,0.000000,0.052588,0.095255,0.239470,1.0
2,0.589439,0.718481,0.622581,0.677858,1.000000,0.879442,0.795931,0.746743,0.701039,0.504959,...,0.561423,0.461706,0.363009,0.074488,0.000000,0.052588,0.095255,0.239470,0.380014,0.0
3,0.718481,0.622581,0.677858,1.000000,0.879442,0.795931,0.746743,0.701039,0.504959,0.497799,...,0.461706,0.363009,0.074488,0.000000,0.052588,0.095255,0.239470,0.380014,0.377605,1.0
4,0.622581,0.677858,1.000000,0.879442,0.795931,0.746743,0.701039,0.504959,0.497799,0.595000,...,0.363009,0.074488,0.000000,0.052588,0.095255,0.239470,0.380014,0.377605,0.384534,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1742180,0.000000,0.067683,0.194752,0.283696,0.214265,0.204720,0.277216,0.381458,0.433030,0.384355,...,0.691120,0.734876,0.852042,0.899713,0.959213,1.000000,0.962991,0.700302,0.634353,0.0
1742181,0.000000,0.136294,0.231695,0.157223,0.146985,0.224745,0.336554,0.391870,0.339662,0.502170,...,0.715629,0.841301,0.892432,0.956252,1.000000,0.960305,0.678545,0.607808,0.358121,1.0
1742182,0.000000,0.110455,0.024232,0.012378,0.102408,0.231861,0.295906,0.235459,0.423611,0.631388,...,0.816258,0.875458,0.949349,1.000000,0.954041,0.627818,0.545920,0.256832,0.351712,0.0
1742183,0.099306,0.012002,0.000000,0.091158,0.222234,0.287081,0.225877,0.416387,0.626768,0.611608,...,0.873897,0.948714,1.000000,0.953465,0.623154,0.540229,0.247518,0.343587,0.189174,1.0


In [9]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")

Random Forest :  0.4971751412429379
Logistic Regression: 0.5819209039548022
Nearest Centroid: 0.5141242937853108
GaussianNB: 0.5310734463276836
K-Neighbors: 0.5254237288135594
Decision Tree: 0.4689265536723164
Gradient Boost: 0.536723163841808
SVM: 0.5310734463276836


In [38]:
model = Sequential()
model.add(Dense(2500, activation='relu', kernel_initializer='he_normal', input_shape=(xTrain.shape[1],)))
model.add(Dense(1500, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1000, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(500, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 2500)              55000     
                                                                 
 dense_17 (Dense)            (None, 1750)              4376750   
                                                                 
 dense_18 (Dense)            (None, 1000)              1751000   
                                                                 
 dense_19 (Dense)            (None, 500)               500500    
                                                                 
 dense_20 (Dense)            (None, 1)                 501       
                                                                 
Total params: 6,683,751
Trainable params: 6,683,751
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.fit(xTrain,yTrain,epochs=15,batch_size=140,validation_data=(xTest,yTest))

Epoch 1/11
11200/11200 [==============================] - 1754s 157ms/step - loss: 0.6645 - accuracy: 0.5789 - val_loss: 0.6236 - val_accuracy: 0.6336
Epoch 2/11
11200/11200 [==============================] - 1777s 159ms/step - loss: 0.5891 - accuracy: 0.6665 - val_loss: 0.5592 - val_accuracy: 0.6938
Epoch 3/11
11200/11200 [==============================] - 1751s 156ms/step - loss: 0.5112 - accuracy: 0.7351 - val_loss: 0.4944 - val_accuracy: 0.7515
Epoch 4/11
11200/11200 [==============================] - 1747s 156ms/step - loss: 0.4223 - accuracy: 0.7992 - val_loss: 0.4254 - val_accuracy: 0.8029
Epoch 5/11
11200/11200 [==============================] - 1800s 161ms/step - loss: 0.3450 - accuracy: 0.8473 - val_loss: 0.3672 - val_accuracy: 0.8434
Epoch 6/11
11200/11200 [==============================] - 1751s 156ms/step - loss: 0.2866 - accuracy: 0.8798 - val_loss: 0.3411 - val_accuracy: 0.8617
Epoch 7/11
11200/11200 [==============================] - 1786s 159ms/step - loss: 0.2452 - ac

In [42]:
model.evaluate(xTest,yTest)

5445/5445 [==============================] - 106s 19ms/step - loss: 0.2676 - accuracy: 0.9144


[0.26759475469589233, 0.9144352674484253]

In [43]:
model.save("weights_91_1500_daily_crypto.h5")